In [ ]:
import pandas as pd
import pandas as pd1
import datetime as dt
import numpy as np
import locale

locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

# importing the data
df = pd.read_csv("/content/retail.csv",sep=',',decimal=',') #Clean Price column from comma to dot
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,1.12.2009 07:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,1.12.2009 07:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,1.12.2009 07:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,1.12.2009 07:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.12.2009 07:45,1.25,13085.0,United Kingdom


**Task 1.**

**1.1 Calculating Frequency in RFM Model**

In [ ]:
frequency_df = df.drop_duplicates().groupby(
	by=['Customer ID'], as_index=False)['InvoiceDate'].count()
frequency_df.columns = ['Customer ID', 'Freq']
frequency_df.head()


,Customer ID,Freq
0,12346.0,3
1,12349.0,5
2,12358.0,17
3,12359.0,32
4,12362.0,1


**1.2 Calculating Monetary in RFM Model**

In [ ]:
# Check Datatype
df.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID     object
Country         object
dtype: object

In [ ]:
# Convert Date colum to Date & Time Format from object
df['InvoiceDate']=pd.to_datetime(df.InvoiceDate)
#df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], format='%d.%m.%Y %H:%M')

In [ ]:
df['Total'] = df['Price']*df['Quantity']
monetary_df = df.groupby(by='Customer ID', as_index=False)['Total'].sum() # Total Rev
monetary_df1 = df.groupby(by='Customer ID', as_index=False)['Total'].mean() # Avg Rev
merger = monetary_df.merge(monetary_df1, on='Customer ID')
merger.rename(columns = {'Total_x':'Tot_Rev'}, inplace = True) # Rename cols
merger.rename(columns = {'Total_y':'Avg_Rev'}, inplace = True) # Rename cols
merger.head()


,Customer ID,Tot_Rev,Avg_Rev
0,12346.0,90.00,30.000000
1,12349.0,-24.15,-4.830000
2,12358.0,1429.83,84.107647
3,12359.0,760.69,23.771563
4,12362.0,130.00,130.000000


In [ ]:
from datetime import datetime
df_recency = df.groupby(by='Customer ID',
						as_index=False)['InvoiceDate'].max()
df_recency.columns = ['Customer ID', 'Duration']
df_recency['Duration'] = pd.to_datetime(df_recency['Duration'], format='%d.%m.%Y %H:%M')
recent_date = "1.1.2012"
df_recency['handling_time'] = datetime.strptime(recent_date, "%d.%m.%Y") - df_recency['Duration']
#print(df_recency)
#drop Duration column now
del df_recency['Duration']
#df_recency.dtypes
df_recency.rename(columns={'handling_time': 'Duration'}, inplace=True)
print(df_recency)

    Customer ID           Duration
0       12346.0  747 days 12:58:00
1       12349.0  993 days 11:11:00
2       12358.0  871 days 16:01:00
3       12359.0  963 days 10:28:00
4       12362.0 1083 days 13:50:00
..          ...                ...
948     18271.0  871 days 12:33:00
949     18275.0  871 days 07:37:00
950     18276.0  810 days 06:46:00
951     18286.0  745 days 13:15:00
952     18287.0 1083 days 09:41:00

[953 rows x 2 columns]


In [ ]:
#Merge 2 Pandas Frequency and Duration
#pd.concat([df_recency, frequency_df]).drop_duplicates()
merged_df = df_recency.merge(frequency_df, on='Customer ID', how='outer').drop_duplicates()
print(merged_df)


    Customer ID           Duration  Freq
0       12346.0  747 days 12:58:00     3
1       12349.0  993 days 11:11:00     5
2       12358.0  871 days 16:01:00    17
3       12359.0  963 days 10:28:00    32
4       12362.0 1083 days 13:50:00     1
..          ...                ...   ...
948     18271.0  871 days 12:33:00     9
949     18275.0  871 days 07:37:00     9
950     18276.0  810 days 06:46:00    16
951     18286.0  745 days 13:15:00    17
952     18287.0 1083 days 09:41:00     1

[953 rows x 3 columns]


In [ ]:
#Merge with another Pandas
merged_df1 = merged_df.merge(merger, on='Customer ID', how='outer').drop_duplicates()
print(merged_df1)

    Customer ID           Duration  Freq  Tot_Rev     Avg_Rev
0       12346.0  747 days 12:58:00     3    90.00   30.000000
1       12349.0  993 days 11:11:00     5   -24.15   -4.830000
2       12358.0  871 days 16:01:00    17  1429.83   84.107647
3       12359.0  963 days 10:28:00    32   760.69   23.771563
4       12362.0 1083 days 13:50:00     1   130.00  130.000000
..          ...                ...   ...      ...         ...
948     18271.0  871 days 12:33:00     9   192.55   21.394444
949     18275.0  871 days 07:37:00     9   372.64   41.404444
950     18276.0  810 days 06:46:00    16   218.05   13.628125
951     18286.0  745 days 13:15:00    17   462.95   27.232353
952     18287.0 1083 days 09:41:00     1    -5.10   -5.100000

[953 rows x 5 columns]


In [ ]:
final = merged_df1.sort_values(by='Customer ID', ascending=False)
print(final)

    Customer ID           Duration  Freq  Tot_Rev     Avg_Rev
952     18287.0 1083 days 09:41:00     1    -5.10   -5.100000
951     18286.0  745 days 13:15:00    17   462.95   27.232353
950     18276.0  810 days 06:46:00    16   218.05   13.628125
949     18275.0  871 days 07:37:00     9   372.64   41.404444
948     18271.0  871 days 12:33:00     9   192.55   21.394444
..          ...                ...   ...      ...         ...
4       12362.0 1083 days 13:50:00     1   130.00  130.000000
3       12359.0  963 days 10:28:00    32   760.69   23.771563
2       12358.0  871 days 16:01:00    17  1429.83   84.107647
1       12349.0  993 days 11:11:00     5   -24.15   -4.830000
0       12346.0  747 days 12:58:00     3    90.00   30.000000

[953 rows x 5 columns]


In [ ]:
df.head()
df.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID            object
Country                object
Total                 float64
dtype: object

**Task 2**
### To find out for which ranges of recency, is the company realizing around 75% of the total revenue.
---



In [ ]:
Recency_Deciles = pd.qcut(x = final['Duration'], q = 10).reset_index()\
.rename(columns = {'Duration':'Recency_Deciles'})

In [ ]:
Recency_Deciles.drop(columns = 'index', inplace = True)

In [ ]:
Task_2 = pd.concat([final, Recency_Deciles], axis = 1)

In [ ]:
Task_2 = Task_2.groupby('Recency_Deciles')[['Tot_Rev']].sum()


In [ ]:
Task_2.reset_index(inplace = True)
Task_2

,index,Recency_Deciles,Tot_Rev
0,0,"(745 days 08:46:59.999999999, 746 days 08:44:48]",37748.62
1,1,"(746 days 08:44:48, 747 days 10:41:24]",48941.52
2,2,"(747 days 10:41:24, 779 days 08:41:36]",41803.01
3,3,"(779 days 08:41:36, 810 days 07:37:12]",66971.43
4,4,"(810 days 07:37:12, 840 days 11:08:00]",48498.96
5,5,"(840 days 11:08:00, 871 days 11:49:24]",61466.36
6,6,"(871 days 11:49:24, 932 days 10:53:48]",45466.51
7,7,"(932 days 10:53:48, 1024 days 05:44:12]",94756.44
8,8,"(1024 days 05:44:12, 1052 days 11:53:24]",60020.00
9,9,"(1052 days 11:53:24, 1083 days 14:52:00]",43509.25


In [ ]:
# For the Cumulative_Revenue
Task_2['CumSum_Rev'] = Task_2['Tot_Rev'].cumsum()

# For Total_revenue_across_Deciles
Task_2['Total_Rev_Across_Deciles'] = Task_2['Tot_Rev'].sum()

# For Percentage of Total_Revenue
Task_2['Perc_tot_Rev'] = (Task_2['CumSum_Rev']/Task_2['Total_Rev_Across_Deciles'])

# Final Task_2 Table
Task_2

,index,Recency_Deciles,Tot_Rev,CumSum_Rev,Total_Rev_Across_Deciles,Perc_tot_Rev
0,0,"(745 days 08:46:59.999999999, 746 days 08:44:48]",37748.62,37748.62,549182.1,0.068736
1,1,"(746 days 08:44:48, 747 days 10:41:24]",48941.52,86690.14,549182.1,0.157853
2,2,"(747 days 10:41:24, 779 days 08:41:36]",41803.01,128493.15,549182.1,0.233972
3,3,"(779 days 08:41:36, 810 days 07:37:12]",66971.43,195464.58,549182.1,0.355919
4,4,"(810 days 07:37:12, 840 days 11:08:00]",48498.96,243963.54,549182.1,0.444231
5,5,"(840 days 11:08:00, 871 days 11:49:24]",61466.36,305429.90,549182.1,0.556154
6,6,"(871 days 11:49:24, 932 days 10:53:48]",45466.51,350896.41,549182.1,0.638944
7,7,"(932 days 10:53:48, 1024 days 05:44:12]",94756.44,445652.85,549182.1,0.811485
8,8,"(1024 days 05:44:12, 1052 days 11:53:24]",60020.00,505672.85,549182.1,0.920774
9,9,"(1052 days 11:53:24, 1083 days 14:52:00]",43509.25,549182.10,549182.1,1.000000


**Task 3**
##### To find out within the subset of data identified above, for what ranges of frequency is ~65% of the revenue in the subset realized.

---



In [ ]:
Task_2

,index,Recency_Deciles,Tot_Rev,CumSum_Rev,Total_Rev_Across_Deciles,Perc_tot_Rev
0,0,"(745 days 08:46:59.999999999, 746 days 08:44:48]",37748.62,37748.62,549182.1,0.068736
1,1,"(746 days 08:44:48, 747 days 10:41:24]",48941.52,86690.14,549182.1,0.157853
2,2,"(747 days 10:41:24, 779 days 08:41:36]",41803.01,128493.15,549182.1,0.233972
3,3,"(779 days 08:41:36, 810 days 07:37:12]",66971.43,195464.58,549182.1,0.355919
4,4,"(810 days 07:37:12, 840 days 11:08:00]",48498.96,243963.54,549182.1,0.444231
5,5,"(840 days 11:08:00, 871 days 11:49:24]",61466.36,305429.90,549182.1,0.556154
6,6,"(871 days 11:49:24, 932 days 10:53:48]",45466.51,350896.41,549182.1,0.638944
7,7,"(932 days 10:53:48, 1024 days 05:44:12]",94756.44,445652.85,549182.1,0.811485
8,8,"(1024 days 05:44:12, 1052 days 11:53:24]",60020.00,505672.85,549182.1,0.920774
9,9,"(1052 days 11:53:24, 1083 days 14:52:00]",43509.25,549182.10,549182.1,1.000000


In [ ]:
# Using the qcut function for making Deciles
freq_dec = pd.qcut(x = final['Freq'], q = 10).reset_index()

# Droping the extra index columns
freq_dec.drop(columns = 'index', inplace = True)

# Renaming the column
freq_dec.rename(columns = {'Freq':'Freq_Deciles'}, inplace = True)

In [ ]:
# Concatenating the Frequency Deciles variable with the RFM Table
RFM = pd.concat([final, freq_dec], axis = 1)
RFM

,Customer ID,Duration,Freq,Tot_Rev,Avg_Rev,Freq_Deciles
952,18287.0,1083 days 09:41:00,1,-5.10,-5.100000,"(2.0, 6.0]"
951,18286.0,745 days 13:15:00,17,462.95,27.232353,"(2.0, 6.0]"
950,18276.0,810 days 06:46:00,16,218.05,13.628125,"(13.0, 18.0]"
949,18275.0,871 days 07:37:00,9,372.64,41.404444,"(30.4, 41.6]"
948,18271.0,871 days 12:33:00,9,192.55,21.394444,"(0.999, 2.0]"
...,...,...,...,...,...,...
4,12362.0,1083 days 13:50:00,1,130.00,130.000000,"(6.0, 9.0]"
3,12359.0,963 days 10:28:00,32,760.69,23.771563,"(6.0, 9.0]"
2,12358.0,871 days 16:01:00,17,1429.83,84.107647,"(13.0, 18.0]"
1,12349.0,993 days 11:11:00,5,-24.15,-4.830000,"(13.0, 18.0]"


In [ ]:
# Calculating Total Revenue by Frequency Deciles
Task_3 = RFM.groupby('Freq_Deciles')[['Tot_Rev']].sum()
Task_3.reset_index(inplace = True)

# Calculating the Cumulative Sum of the Total_revenue
Task_3['CumSum'] = Task_3['Tot_Rev'].cumsum()

# Calculating the Total Revenue
Task_3['Total'] = Task_3['Tot_Rev'].sum()

# Calculating the Percentage Value
Task_3['Perc_Total'] = Task_3['CumSum']/Task_3['Total']

In [ ]:
Task_3

,Freq_Deciles,Tot_Rev,CumSum,Total,Perc_Total
0,"(0.999, 2.0]",57026.39,57026.39,549182.1,0.103839
1,"(2.0, 6.0]",52380.70,109407.09,549182.1,0.199218
2,"(6.0, 9.0]",70232.98,179640.07,549182.1,0.327105
3,"(9.0, 13.0]",46163.15,225803.22,549182.1,0.411163
4,"(13.0, 18.0]",51115.55,276918.77,549182.1,0.504239
5,"(18.0, 22.0]",32552.40,309471.17,549182.1,0.563513
6,"(22.0, 30.4]",82211.35,391682.52,549182.1,0.713211
7,"(30.4, 41.6]",48234.87,439917.39,549182.1,0.801041
8,"(41.6, 59.0]",52271.46,492188.85,549182.1,0.896222
9,"(59.0, 422.0]",56993.25,549182.10,549182.1,1.000000
